In [ ]:
!pip3 install tinkoff-investments
!pip3 install yfinance
!pip3 install pandas
!pip3 install numpy
!pip3 install decimal
!pip3 install locale

In [91]:
from tinkoff.invest import Client

TOKEN = 't.b7eKSJEp3fpSiiv4mVt4fWwKIxaMHM1lDMtpGsPTeyl850b9Y4MluXYv-EQrj1vEu7QfkNwqGqGPfTW9N6EvTg'

import pandas as pd
from datetime import timedelta
from datetime import datetime, timedelta, timezone
from decimal import Decimal
from tinkoff.invest import CandleInterval, Client, HistoricCandle, Quotation, SubscriptionInterval
from tinkoff.invest.utils import now

__all__ = (
    "get_intervals",
    "quotation_to_decimal",
    "decimal_to_quotation",
    "candle_interval_to_subscription_interval",
    "now",
    "candle_interval_to_timedelta",
    "ceil_datetime",
    "floor_datetime",
    "dataclass_from_dict",
    "datetime_range_floor",
)


'''MoneyValue — используется для параметров, у которых есть денежный эквивалент. Возьмем для примера стоимость ценных бумаг — тип состоит из трех параметров:
1) currency — строковый ISO-код валюты, например RUB или USD;
2) units — целая часть суммы;
3) nano — дробная часть суммы, миллиардные доли единицы.
'''
# Quotation type = MoneyValue. We need to convert this to decimal in order to fetch price per share
def quotation_to_decimal(quotation: Quotation) -> Decimal:
    fractional = quotation.nano / Decimal("10e8")
    return Decimal(quotation.units) + fractional

def get_final_stock_volumes(_input: int):
    return f'{_input:,} ₽'

def get_final_lots(_lots: int):
    return f'{_lots:,} шт.'

def calculate_net_change(current_closing_price: int, prev_closing_price: int):
    return f'Изменение цены: {round(((current_closing_price - prev_closing_price) / prev_closing_price * 100), 2)}%'

def calculate_net_change_per_day(current_closing_price: int, yesterday_closing_price: int):
    # current price minus 840 indexes in order to fetch price index yesterday for 1 minute candle
    return f'Изменение за день: {round(((current_closing_price - yesterday_closing_price) / yesterday_closing_price * 100), 2)}%'

def make_million_volumes_on_float_stock_prices(price: int):
    price = str(price)
    price += '0000'
    return get_final_stock_volumes(int(price))

#volumes = []
#prices = []
#time = []


def main():
    # vtbr BBG004730ZJ9
    # lkoh BBG004731032
    with Client(TOKEN) as client:
        l1_volumes = []
        l2_lots = []
        l3_prices = []
        l4_time = []
        data = {"Объем": l1_volumes, "Лоты": l2_lots, "Цена": l3_prices, "Время": l4_time}
        threshold = 5.0
        for candle in client.get_all_candles(
            figi="BBG004730ZJ9",
            from_=now() - timedelta(days=90),
            interval=CandleInterval.CANDLE_INTERVAL_1_MIN,
        ):
            if quotation_to_decimal(candle.close) < 1:
                final_stock_volume_rub = int(candle.volume * float(quotation_to_decimal(candle.close)))
                l1_volumes.append(make_million_volumes_on_float_stock_prices(final_stock_volume_rub))
                l2_lots.append(get_final_lots(candle.volume))
                l3_prices.append(float(quotation_to_decimal(candle.close)))
                l4_time.append(candle.time)

                #print(f'ВТБ (VTBR), Время: {candle.time}, {get_final_lots(candle.volume)}, {make_million_volumes_on_float_stock_prices(final_stock_volume_rub)}, Цена: {float(quotation_to_decimal(candle.close))} ₽')


            else:
                final_stock_volume_rub = int(candle.volume * quotation_to_decimal(candle.close))
                l1_volumes.append(final_stock_volume_rub)
                l2_lots.append(get_final_lots(candle.volume))
                l3_prices.append(int(quotation_to_decimal(candle.close)))
                l4_time.append(candle.time)

                #print(f'Лукойл (LKOH), Время: {candle.time}, {get_final_lots(candle.volume)}, {get_final_stock_volumes(final_stock_volume_rub)}, Цена: {float(quotation_to_decimal(candle.close))} ₽')

            
            #volumes.append(final_stock_volume_rub)
            #prices.append(int(quotation_to_decimal(candle.close)))
            #time.append(candle.time)
        
        df = pd.DataFrame(data)

        '''Объем                 97,580,000 ₽
           Лоты                   461,947 шт.
           Цена                      0.021125
           Время    2023-04-27 07:16:00+00:00
           Name: 52227, dtype: object'''
        print(df.iloc[52227])

        # Calculate the rolling average and standard deviation of the trading volume
        '''volume_mean = round(df["Объем"].mean())
        volume_std = round(df["Объем"].std())

        # Calculate the rolling average and standard deviation of the trading prices
        prcies_mean = round(df["Цена"].mean())
        prices_std = round(df["Цена"].std())'''

        '''for i in range(len(df)):
            abnormal_volume = (df["Объем"].iloc[i] - volume_mean) / volume_std
            abnormal_price_changes = (df["Цена"].iloc[i] - prcies_mean) / prices_std
            
            if abnormal_volume > threshold or abnormal_price_changes > threshold:
                print(f'Зафиксирован аномальный объем! ВТБ (VTBR) {get_final_stock_volumes(df["Объем"].iloc[i])},  Цена: {df["Цена"].iloc[i]} ₽, {calculate_net_change(df["Цена"].iloc[i], df["Цена"].iloc[i - 1])}, {df["Время"].iloc[i]}, {calculate_net_change_per_day(df["Цена"].iloc[i], df["Цена"].iloc[i - 840])}')

        #if ((df["Объем"].iloc[-1] - volume_mean) / volume_std) > threshold or ((df["Цена"].iloc[-1] - prcies_mean) / prices_std) > threshold:
         #   print(f'Зафиксирован аномальный объем! Лукойл (LKOH) {get_final_stock_volumes(df["Объем"].iloc[-1])},  Цена: {df["Цена"].iloc[-1]} ₽, {calculate_net_change(df["Цена"].iloc[-1], df["Цена"].iloc[-1 - 1])}, {df["Время"].iloc[-1]}, {calculate_net_change_per_day(df["Цена"].iloc[-1], df["Цена"].iloc[-1 - 840])}')'''

        
    return 0

if __name__ == "__main__":
    main()


Объем                 97,580,000 ₽
Лоты                   461,947 шт.
Цена                      0.021125
Время    2023-04-27 07:16:00+00:00
Name: 52227, dtype: object


In [ ]:
def get_final_stock_volumes(_input):
    return f'Объем: {_input:,} ₽'

def get_final_lots(_lots):
    return f'Кол-во лотов: {_lots:,} шт.'

# 59706


price = 4666.00000000
lots = 1085
print(f'Цена: {price} ₽')
print(get_final_lots(lots))
output = int(price * lots)
print(output)
print(get_final_stock_volumes(output))


In [ ]:
#data = {'Время': ['2023-05-10 13:52:00+00:00', '2023-05-10 13:52:00+00:00'], 'Кол-во лотов': [380, 832852], 'Объем': [2134324, 23455], 'Цена': [4647.00000000, 4645.50000000]}
dataframe = {'Объем': volumes, 'Цена': prices, 'Время': time}
df = pd.DataFrame(dataframe)
threshold = 5.0

# Calculate the rolling average and standard deviation of the trading volume
volume_mean = round(df['Объем'].mean())
volume_std = round(df['Объем'].std())

# Calculate the rolling average and standard deviation of the trading prices
prcies_mean = round(df['Цена'].mean())
prices_std = round(df['Цена'].std())

#abnormal_volume = (df['Объем'].iloc[-1] - volume_mean) / volume_std

def calculate_net_change(current_closing_price: int, prev_closing_price: int):
    return f'Изменение цены: {round(((current_closing_price - prev_closing_price) / prev_closing_price * 100), 2)}%'

def calculate_net_change_per_day(current_closing_price: int, yesterday_closing_price: int):
    # current price minus 840 indexes in order to fetch price index yesterday for 1 minute candle
    return f'Изменение за день: {round(((current_closing_price - yesterday_closing_price) / yesterday_closing_price * 100), 2)}%'

for i in range(len(df)):
    abnormal_volume = (df['Объем'].iloc[i] - volume_mean) / volume_std
    abnormal_price_changes = (df['Цена'].iloc[i] - prcies_mean) / prices_std
    #print(abnormal_volume, abnormal_price_changes)

    if abnormal_volume > threshold or abnormal_price_changes > threshold:
        print(f'Зафиксирован аномальный объем! Лукойл (LKOH) {get_final_stock_volumes(df["Объем"].iloc[i])},  Цена: {df["Цена"].iloc[i]} ₽, {calculate_net_change(df["Цена"].iloc[i], df["Цена"].iloc[i - 1])}, {df["Время"].iloc[i]}, {calculate_net_change_per_day(df["Цена"].iloc[i], df["Цена"].iloc[i - 840])}')

In [ ]:
import asyncio
import logging
import os
from datetime import timedelta
from typing import List, Optional

from tinkoff.invest import AioRequestError, AsyncClient, CandleInterval, HistoricCandle
from tinkoff.invest.async_services import AsyncServices
from tinkoff.invest.utils import now

TOKEN = 't.b7eKSJEp3fpSiiv4mVt4fWwKIxaMHM1lDMtpGsPTeyl850b9Y4MluXYv-EQrj1vEu7QfkNwqGqGPfTW9N6EvTg'


logging.basicConfig(format="%(asctime)s %(levelname)s:%(message)s", level=logging.DEBUG)
logger = logging.getLogger(__name__)


class LogOnlyCandlesStrategy:
    """This class is responsible for a strategy. You can put here
    your methods for your strategy."""

    def __init__(
        self,
        figi: str,
        timeframe: CandleInterval,
        days_back: int,
        check_interval: int,
        client: Optional[AsyncServices],
    ):
        self.account_id = None
        self.figi = figi
        self.timeframe = timeframe
        self.days_back = days_back
        self.check_interval = check_interval
        self.client = client
        self.candles: List[HistoricCandle] = []

    async def get_historical_data(self):
        """
        Gets historical data for the instrument. Returns list of candles.
        Requests all the candles of timeframe from days_back to now.

        :return: list of HistoricCandle
        """
        logger.debug(
            "Start getting historical data for %s days back from now. figi=%s",
            self.days_back,
            self.figi,
        )
        async for candle in self.client.get_all_candles(
            figi=self.figi,
            from_=now() - timedelta(days=self.days_back),
            to=now(),
            interval=self.timeframe,
        ):
            if candle not in self.candles:
                if candle.is_complete:
                    self.candles.append(candle)
                    logger.debug("Found %s - figi=%s", candle, self.figi)

    async def ensure_market_open(self):
        """
        Ensure that the market is open. Loop until the instrument is available.
        :return: when instrument is available for trading
        """
        trading_status = await self.client.market_data.get_trading_status(
            figi=self.figi
        )
        while not (
            trading_status.market_order_available_flag
            and trading_status.api_trade_available_flag
        ):
            logger.debug("Waiting for the market to open. figi=%s", self.figi)
            await asyncio.sleep(60)
            trading_status = await self.client.market_data.get_trading_status(
                figi=self.figi
            )

    async def main_cycle(self):
        """Main cycle for live strategy."""
        while True:
            try:
                await self.ensure_market_open()
                await self.get_historical_data()

                # put your strategy code here for live
                # to generate signals for buying or selling tickers
                logger.debug(
                    "- live mode: run some strategy code to buy or sell - figi=%s",
                    self.figi,
                )

            except AioRequestError as are:
                logger.error("Client error %s", are)

            await asyncio.sleep(self.check_interval)

    async def start(self):
        """Strategy starts from this function."""
        if self.account_id is None:
            try:
                self.account_id = (
                    (await self.client.users.get_accounts()).accounts.pop().id
                )
            except AioRequestError as are:
                logger.error("Error taking account id. Stopping strategy. %s", are)
                return
        await self.main_cycle()


async def run_strategy(portfolio, timeframe, days_back, check_interval):
    """From this function we are starting
    strategy for every ticker from portfolio.
    """
    async with AsyncClient(token=TOKEN, app_name="TinkoffApp") as client:
        strategy_tasks = []
        for instrument in portfolio:
            strategy = LogOnlyCandlesStrategy(
                figi=instrument,
                timeframe=timeframe,
                days_back=days_back,
                check_interval=check_interval,
                client=client,
            )
            strategy_tasks.append(asyncio.create_task(strategy.start()))
        await asyncio.gather(*strategy_tasks)


if __name__ == "__main__":

    vtbr_figi = "BBG004730ZJ9"
    portfolio = {vtbr_figi}
    timeframe = CandleInterval.CANDLE_INTERVAL_1_MIN
    days_back = 1
    check_interval = 10  # seconds to check interval for new completed candle

    loop = asyncio.get_event_loop()
    task = loop.create_task(
        run_strategy(
            portfolio=portfolio,
            timeframe=timeframe,
            days_back=days_back,
            check_interval=check_interval,
        )
    )
    loop.run_until_complete(task)
